In [89]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from keras.optimizers import Adam

In [90]:
# drive path
drive = '/content/drive/My Drive/Colab Notebooks/HAR'

# local path
# local = 'G:/My Drive/Colab Notebooks/HAR'
local = 'G:/HAR'

hapt = '/HAPT Dataset'       # Human Activity Postural Transitions data set
har = '/UCI HAR Dataset'      # Human Activity Recognition data set

path_drive = drive + har
path_local = local + har

In [91]:
using_drive = 1

if using_drive == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  path = path_drive
else:
  path = path_local

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Gravitational acceleration data files for x, y and z axes: `total_acc_x_train.txt`, `total_acc_y_train.txt` , `total_acc_z_train.txt`.<br>
Body acceleration data files for x, y and z axes: `body_acc_x_train.txt`, `body_acc_y_train.txt`, `body_acc_z_train.txt`.<br>
Body gyroscope data files for x, y and z axes: `body_gyro_x_train.txt`,
`body_gyro_y_train.txt`, `body_gyro_z_train.txt`.<br>
The structure is mirrored in the test directory.

In [92]:
# load a single file as a numpy array
def load_file(filepath):
  dataframe = read_csv(filepath, header=None, delim_whitespace=True, encoding='utf-8', encoding_errors='ignore')
  return dataframe.values

# load a list of files, such as x, y, z data for a given variable
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data)
  # stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + '/' + group + '/Inertial Signals/'
  # load all 9 files as a single array
  filenames = list()
  # total acceleration
  filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
  # body acceleration
  filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
  # body gyroscope
  filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  # y = load_file(prefix + '/' + group + '/y_' + group + '.txt') # doesn't work with default 'utf-8'
  y = read_csv(prefix + '/' + group + '/y_' + group + '.txt', header=None, encoding='utf-16')
  return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(path=''):
  # print(path)
  # load all train
  trainX, trainy = load_dataset_group('train', path)
  # load all test
  testX, testy = load_dataset_group('test', path)
  # zero-offset class values
  trainy = trainy - 1
  testy = testy - 1
  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)
  return trainX, trainy, testX, testy

In [93]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 10, 32 # default 0, 10, 32
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  dropout = 0.5 # default 0.5
  units = 100 # default 100
  dense_layer_nodes = 100 # default 100
  model = Sequential()
  model.add(LSTM(units, dropout=dropout, return_sequences=True, input_shape=(n_timesteps,n_features)))
  model.add(LSTM(units, dropout=dropout, return_sequences=True))
  model.add(LSTM(units, dropout=dropout, return_sequences=True))
  model.add(LSTM(units, dropout=dropout, return_sequences=True))
  model.add(LSTM(units, dropout=dropout, return_sequences=True))
  model.add(LSTM(units, dropout=dropout))
  # model.add(Dropout(dropout))
  model.add(Dense(dense_layer_nodes))
  model.add(Dense(n_outputs, activation='softmax'))
  opt = Adam(learning_rate=0.001) # default 0.001 = 1e-3
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy, model

In [94]:
# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [95]:
# run an experiment
def run_experiment(repeats=10, path=''):
  print("Experiment starting. Loading dataset...")
  # load data
  trainX, trainy, testX, testy = load_dataset(path)
  print("Finished loading dataset, starting model training...")
  # repeat experiment
  scores = list()
  for r in range(repeats):
    score, model = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)
  print(model.summary())

In [96]:
# run the experiment
run_experiment(path=path)

Experiment starting. Loading dataset...
Finished loading dataset, starting model training...
>#1: 86.664
>#2: 92.026
>#3: 90.533
>#4: 90.092
>#5: 90.024
>#6: 91.415
>#7: 88.599
>#8: 87.581
>#9: 90.533
>#10: 55.955
[86.6644024848938, 92.02578663825989, 90.53274393081665, 90.09161591529846, 90.0237500667572, 91.41499996185303, 88.59857320785522, 87.5805914402008, 90.53274393081665, 55.95520734786987]
Accuracy: 86.342% (+/-10.251)
Model: "sequential_119"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_474 (LSTM)             (None, 128, 100)          44000     
                                                                 
 lstm_475 (LSTM)             (None, 128, 100)          80400     
                                                                 
 lstm_476 (LSTM)             (None, 128, 100)          80400     
                                                                 
 lstm_477 (LSTM)